In [2]:
import os
import random
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SiameseNetwork(nn.Module):
    def __init__(self, input_size=(1, 128, 128)):
        super(SiameseNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )

        # Compute flattened size dynamically
        with torch.no_grad():
            dummy_input = torch.zeros(1, *input_size)
            dummy_output = self.cnn(dummy_input)
            flattened_size = dummy_output.view(1, -1).size(1)

        self.fc = nn.Sequential(
            nn.Linear(flattened_size, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 128)
        )

    def forward_once(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    def forward(self, x1, x2):
        out1 = self.forward_once(x1)
        out2 = self.forward_once(x2)
        return out1, out2



In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNetwork(input_size=(1, 128, 128)).to(device)
model.load_state_dict(torch.load("siamese_signature.pth", map_location=device))
model.eval()


C:\Users\Gopi\AppData\Local\Temp\ipykernel_30088\2806332792.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("siamese_signature.pth", map

SiameseNetwork(
  (cnn): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU(inplace=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=21632, out_features=512, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=512, out_features=128, bias=True)
  )
)

In [6]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [16]:
from PIL import Image

# Path to reference genuine signature
genuine_path = "113.jpg"  # example
# Path to the new signature to test
test_path = "113.jpg"

# Load and transform
genuine_img = transform(Image.open(genuine_path).convert("L")).unsqueeze(0).to(device)
test_img = transform(Image.open(test_path).convert("L")).unsqueeze(0).to(device)


In [17]:
import torch.nn.functional as F

with torch.no_grad():
    output1, output2 = model(genuine_img, test_img)
    euclidean_distance = F.pairwise_distance(output1, output2)
    print("Distance:", euclidean_distance.item())


Distance: 1.1313708455418237e-05


In [18]:
threshold = 1.0  # you can tune this based on validation set
if euclidean_distance.item() < threshold:
    print("Genuine Signature ✅")
else:
    print("Forged Signature ❌")


Genuine Signature ✅
